# Topic Modeling: Descubriendo Temáticas en los Comentarios de Facebook

## Cargar el archivo
Se extrajeron 1,569 comentarios de septiembre y octubre año 2020 en los que se menciona a CitiBanamex

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import string
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
#nltk.download('stopwords')
#nltk.download('punkt')
stop_words_sp = set(stopwords.words('spanish'))

In [ ]:
df = pd.read_csv('../data/BanamexFace.csv')
df.head()

#### Limpieza del texto (puntuación, tokenización y eliminado de stopwords en español)

In [ ]:
stemmer = SnowballStemmer("spanish")
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stop_words_sp]
    return text

In [ ]:
df["Tokens"] = df.com.apply(clean_text)
df.head()

####  Creamos el diccionario necesario para el modelo LDA

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import random
from wordcloud import WordCloud

In [ ]:
diccionario = Dictionary(df.Tokens)
print(f'Número de tokens: {len(diccionario)}')

Con filter_extremes mantenemos únicamentos los tokens que están mantenidos en al menos no_below documentos y los que están contenidos en no mas de no_above documentos. En este último caso le indicamos una fracción del tamaño del corpus.

In [ ]:
diccionario.filter_extremes(no_below=2, no_above = 0.9)
print(f'Número de tokens: {len(diccionario)}')

Generamos una representación vectorial de los documentos

In [ ]:
corpus = [diccionario.doc2bow(noticia) for noticia in df.Tokens]

print(corpus[6])

#### Generamos el modelo LDA

In [ ]:
lda = LdaModel(corpus=corpus, id2word=diccionario, 
               num_topics=40, random_state=42, 
               chunksize=1000, passes=10, alpha='auto')

In [ ]:
topicos = lda.print_topics(num_words=8, num_topics=15)
for topico in topicos:
    print(topico)

Wordcloud de los Tópicos

In [ ]:
for i in range(1, 7):
    plt.figure()
    plt.imshow(WordCloud(background_color='white', prefer_horizontal=1.0)
               .fit_words(dict(lda.show_topic(i, 20))))
    plt.axis("off")
    plt.title("Tópico " + str(i))
    plt.show()

#### Ahora podemos clasificar nuevos comentarios utilizando el modelo LDA generado

In [ ]:
dfs = pd.read_csv('../data/BanamexFaceNew.csv')
dfs.head()

In [ ]:
dfs["Tokens"] = dfs.com.apply(clean_text)
dfs.head()

Identifcando los Tópicos

In [ ]:
bow_articulo_nuevo = diccionario.doc2bow(dfs.Tokens[0])

In [ ]:
# Indices de los topicos mas significativos
dist_indices = [topico[0] for topico in lda[bow_articulo_nuevo]]
# Contribucion de los topicos mas significativos
dist_contrib = [topico[1] for topico in lda[bow_articulo_nuevo]]

In [ ]:
distribucion_topicos = pd.DataFrame({'Topico':dist_indices,'Contribucion':dist_contrib })
distribucion_topicos.sort_values('Contribucion', ascending=False, inplace=True)
ax = distribucion_topicos.plot.bar(y='Contribucion',x='Topico', rot=0, color="green",
                                   title = "Tópicos más importantes para documento nuevo")

Examinamos los Tópicos

In [ ]:
for ind, topico in distribucion_topicos.iterrows():
    print("*** Tópico: " + str(int(topico.Topico)) + " ***")
    palabras = [palabra[0] for palabra in lda.show_topic(topicid=int(topico.Topico))]
    palabras = ', '.join(palabras)
    print(palabras, "\n")

Guardamos el modelo y el diccionario.

In [ ]:
lda.save("comentarios.model")
diccionario.save("comentarios.dictionary")

Elaborado por Luis Cajachahua bajo licencia MIT (2022)